# Covid Proof Table Positioning

In [1]:
# We base ourselves on the paper of Goncalves et al. at:
# https://doi.org/10.1016/j.ejor.2020.04.028
# This is a MIP formulation, which we will convert to a QUBO formulation for execution on a dwave machine.
# A QUBO is a Quadratic Unconstrainted Binary Optimisation problem. 
# All variables are binaries,
# (or sometime integers composed out of binaries, encoded in a particular way (one hot or as an unsigned int, etc.))
# They occur either as linear terms or quadratic terms (but note that x = x^2 for binary x).
# They are unconstrained, so only the objective function has to be formulated.
# When transforming from a MIP with constraints, the constraints have to be converted 
# into (weight balanced) terms in the objective function.
# For this conversion, we use the pyqubo library, at https://pyqubo.readthedocs.io/en/latest/
# which was written by recruit-communications. The resulting qubo datastructure 
# can be sent to a dwave computer to be solved.
# Some principles of the basics of the conversion principles of the pyqubo library are explained at:
# http://www.optimization-online.org/DB_FILE/2018/11/6923.pdfimport math

# The objective function of the MIP is: 
# sum_{i=1}^N   sum_{t=1}^{T}  V^t I_{i}^{t}, 
# where T is the set of table types,
# N^t is the nymber of table of type t,
# N is the total number of tables of any (generic) type (N = max_t{N^t})
# V^t is the value of each placed table of type t 
# and I_{i}^{t} is the boolean variable that is 1 if table i is of type t and placed in the room, otherwise it's 0.

In [2]:
import math 
import pprint

cfg = { 'room': {'dy':10, 'dx': 14},
        'rectangle_types': {'short': {'dy': 1.2, 'dx':2.2, 'value':2.0, 
                                  'max_number':5, 'angles':[0, 90]}, # 1st table type
                            #'long': {'dy': 0.8, 'dx':3.0, 'value':4.0, 
                            #     'max_number':3, 'angles':[0, 90]}, # 2nd table type
                           },
       }

# we reduce the problem to stay below 119 qubits.
cfg = { 'room': {'dy':5, 'dx': 2},
        'rectangle_types': {'short': {'dy': 1.2, 'dx':2.2, 'value':2.0, 
                                  'max_number':2, 'angles':[0, 90]}, # 1st table type
                            #'long': {'dy': 0.8, 'dx':3.0, 'value':4.0, 
                            #     'max_number':3, 'angles':[0, 90]}, # 2nd table type
                           },
       }

In [3]:
resolution_in_m = 0.1  
# The finer the resolution, the more bits needed, so this will be one control parameter
# that allows to scale the needed bits up or down when possible resp. needed.
n_x_steps = math.ceil(cfg['room']['dx'] / resolution_in_m)
n_y_steps = math.ceil(cfg['room']['dy'] / resolution_in_m)
print('n_x_steps={:d} is the number of steps for the room-range horizontal x coordinate variable of each table.'.format(n_x_steps))
print('n_y_steps={:d} is the number of steps for the room-range vertical y coordinate variable of each table.'.format(n_y_steps))
#n_bits_y = math.ceil(math.log(n_y_steps, 2))
#n_bits_x = math.ceil(math.log(n_x_steps, 2))
#print(n_y_steps, n_bits_y, n_x_steps, n_bits_x)

n_x_steps=20 is the number of steps for the room-range horizontal x coordinate variable of each table.
n_y_steps=50 is the number of steps for the room-range vertical y coordinate variable of each table.


In [4]:
from pyqubo import LogEncInteger, Binary, Constraint

import dimod

# room parameters:
W = cfg['room']['dx']
H = cfg['room']['dy']

# table type parameters:
Wt = {}
Ht = {}
Vt = {}
Nt = {}
N = 0
Wt_max = 0
Ht_max = 0
max_mip_obj = 0
for (t, (_, table_type)) in enumerate(cfg['rectangle_types'].items()):
    Wt[t] = table_type['dx']
    Ht[t] = table_type['dy']
    Vt[t] = table_type['value']
    Nt[t] = table_type['max_number']
    N += Nt[t]
    Wt_max = max(Wt_max, Wt[t])  # TODO: Not doing anything with 90 degree rotation yet.
    Ht_max = max(Ht_max, Ht[t])  # TODO: Not doing anything with 90 degree rotation yet.
    max_mip_obj += Nt[t] * Vt[t]  # supposing all tables can fit.
print('N={:d} is the maximum number of potentially placeble tables for all table types summed together.'.format(N))
print('Wt_max={:f} is the maximum width amongst all table types.'.format(Wt_max))
print('Ht_max={:f} is the maximum width amongst all table types.'.format(Ht_max))
print('max_mip_obj={:f} is (an upperbound for) the maximum value the objective could ever reach.'.format(max_mip_obj))
P = max_mip_obj * 10   # we set the penaly P to this value so that hard MIP constraints,
# present in the QUBO model as terms in the QUBO objective to be minimised, are more highly weighted that the
# value the objective can reach.
print('P={:f} is the penalty weight factor we weight all MIP constraints (so other QUBO obj terms) with.'.format(P))


# Create the problem variables.
# We call the Goncalves paper 'I' boolean variables 'b' here instead.
b = {}  
# The rest of the naming is consistent with the Goncalves paper.
x = {}
y = {}
        
n_w_steps = math.ceil(Wt_max / resolution_in_m)
n_h_steps = math.ceil(Ht_max / resolution_in_m)
print('n_w_steps={:d} is the number of steps for the table-range horizontal w size variable of each table.'.format(n_w_steps))
print('n_h_steps={:d} is the number of steps for the table_range vertical h size variable of each table.'.format(n_h_steps))

N=2 is the maximum number of potentially placeble tables for all table types summed together.
Wt_max=2.200000 is the maximum width amongst all table types.
Ht_max=1.200000 is the maximum width amongst all table types.
max_mip_obj=4.000000 is (an upperbound for) the maximum value the objective could ever reach.
P=40.000000 is the penalty weight factor we weight all MIP constraints (so other QUBO obj terms) with.
n_w_steps=22 is the number of steps for the table-range horizontal w size variable of each table.
n_h_steps=12 is the number of steps for the table_range vertical h size variable of each table.


In [5]:
# Create all the variables.

p = {}  # (i,j) bin
q = {}  # (i,j) bin
w = {}  # (i) int
h = {}  # (i) int
s2x = {}  # (i,j) int
s3x = {}  # (i,j) int
s4y = {}  # (i,j) int
s5y = {}  # (i,j) int
s10xL = {} # (i) int
s10xR = {} # (i) int
s11yL = {} # (i) int
s11yR = {} # (i) int
s12 = {}  # (i) bin

for i in range(N):
    for (t, (table_type_name, table_type)) in enumerate(cfg['rectangle_types'].items()):
        b[(i,t)] = Binary('b_{:d}_{:d}'.format(i,t))
        
    x[i] = LogEncInteger('x_{:d}'.format(i), (0, n_x_steps-1))
    y[i] = LogEncInteger('y_{:d}'.format(i), (0, n_y_steps-1))
    w[i] = LogEncInteger('w_{:d}'.format(i), (0, n_w_steps-1))
    h[i] = LogEncInteger('h_{:d}'.format(i), (0, n_h_steps-1))
    s10xL[i] = LogEncInteger('s10xL_{:d}'.format(i), (0, n_x_steps-1))
    s10xR[i] = LogEncInteger('s10xR_{:d}'.format(i), (0, n_x_steps-1))
    s11yL[i] = LogEncInteger('s11yL_{:d}'.format(i), (0, n_y_steps-1))
    s11yR[i] = LogEncInteger('s11yR_{:d}'.format(i), (0, n_y_steps-1))
    s12[i] = Binary('s12_{:d}'.format(i))

    for j in range(i+1, N):
        assert i < j
        p[(i,j)] = Binary('p_{:d}_{:d}'.format(i,j))
        q[(i,j)] = Binary('q_{:d}_{:d}'.format(i,j))   
        s2x[(i,j)] = LogEncInteger('s2x_{:d}_{:d}'.format(i,j), (0, n_x_steps-1))
        s3x[(i,j)] = LogEncInteger('s3x_{:d}_{:d}'.format(i,j), (0, n_x_steps-1))
        s4y[(i,j)] = LogEncInteger('s4y_{:d}_{:d}'.format(i,j), (0, n_y_steps-1))
        s5y[(i,j)] = LogEncInteger('s5y_{:d}_{:d}'.format(i,j), (0, n_y_steps-1))
        
s15 = {} # (t) int
for (t, (_, _)) in enumerate(cfg['rectangle_types'].items()):
    s15[t] = LogEncInteger('s15_{:d}'.format(t), (0, Nt[t]-1))

In [6]:
#print(b)

In [7]:
#print(x)

In [8]:
#print(y)

In [9]:
# Build up original MIP objective (without terms for (energy terms of) constraints).
Ham = 0
for i in range(N):
    for (t, (table_type_name, table_type)) in enumerate(cfg['rectangle_types'].items()):
        Ham += b[(i,t)] * Vt[t]

# Complete the objective from MIP objective to QUBO objective.
for i in range(N):
    lin12 = 1 - s12[i]  # eq (12) 1 - [slack variable] for inequality
    lin13 = w[i]  # eq(13) lhs
    lin14 = h[i]  # eq(14) lhs
    for j in range(i+1, N):
        assert i < j
        Ham += P * Constraint(( x[i] - x[j] + W * (  p[i,j] +   q[i,j]) - s2x[(i,j)] - (w[i]+w[j]) / 2.0 ) ** 2,
                              label='eq2_{:d}_{:d}'.format(i,j)) # eq (2)
        Ham += P * Constraint(( x[j] - x[i] + W * (1-p[i,j] +   q[i,j]) - s3x[(i,j)] - (w[i]+w[j]) / 2.0 ) ** 2,
                              label='eq3_{:d}_{:d}'.format(i,j))  # eq (3)
        
        Ham += P * Constraint(( y[i] - y[j] + H * (  p[i,j] + 1-q[i,j]) - s4y[(i,j)] - (h[i]+h[j]) / 2.0 ) ** 2,
                              label='eq4_{:d}_{:d}'.format(i,j))  # eq (4)
        Ham += P * Constraint(( y[j] - y[i] + H * (1-p[i,j] + 1-q[i,j]) - s5y[(i,j)] - (h[i]+h[j]) / 2.0 ) ** 2,
                              label='eq5_{:d}_{:d}'.format(i,j))  # eq (5)
        
    Ham += P * Constraint((w[i]/2 + s10xL[i] - x[i]) ** 2,
                          label='eq10L_{:d}'.format(i))  # eq (10 left), for all i
    Ham += P * Constraint((x[i] +  s10xR[i] - W + w[i]/2) ** 2,
                          label='eq10R_{:d}'.format(i))  # eq (10 right), for all i
    
    Ham += P * Constraint((h[i]/2 + s11yL[i] - y[i]) ** 2,
                          label='eq11R_{:d}'.format(i))  # eq (11 left), for all i
    Ham += P * Constraint((y[i] + s11yR[i] - H + h[i]/2) ** 2,
                          label='eq11R_{:d}'.format(i))  # eq (11 right), for all i
    
    for (t, (_, _)) in enumerate(cfg['rectangle_types'].items()):
        lin12 -= b[(i,t)]  # eq (12)
        lin13 -= b[(i,t)] * Wt[t]  # eq (13) rhs
        lin14 -= b[(i,t)] * Ht[t]  # eq (14) rhs

    Ham += P * Constraint(lin12 ** 2, label='eq12_{:d}'.format(i))
    # eq (12): so this lin term is pos & minimal when *at most* one b[(i,t)] is 1. 
    # if none of them is, then s(i,t) must be 1 (at optimality), for all i
    
    Ham += P * Constraint(lin13 ** 2, label='eq13_{:d}'.format(i))
    # eq (13): so this lin term is pos & minimal when w[t] = sum_t b[(i,t)]*Wt[t], for all i
                          
    Ham += P * Constraint(lin14 ** 2, label='eq14_{:d}'.format(i))
    # eq (14): so this lin term is pos & minimal when h[t] = sum_t b[(i,t)]*Ht[t], for all i

for (t, (_, _)) in enumerate(cfg['rectangle_types'].items()):
    lin15 = Nt[t] - s15[t]
    for i in range(N):
        lin15 -= b[(i,t)]
    Ham += P *  Constraint(lin15 ** 2, label='eq15_{:d}'.format(t))
    # eq (15) so this lin term is pos & minimal when Nt[t] = s15[t] + sum_i b[(i,t)], for all t
    
#print(Ham)

In [10]:
model = Ham.compile()

In [11]:
#qubo, offset = model.to_qubo()  # why would we need this?

# solve problem:
solve_on_remote_dwave_machine = False

if not solve_on_remote_dwave_machine:  # then solve on this machine locally
    import neal
    sampler = neal.SimulatedAnnealingSampler()
else:  # solve on a real dwave quantum computer
    from dwave.cloud import Client
    client = Client.from_config(token='DEV-????????????????????????????????????????')  # do not expose this on web
    client.get_solvers()
    print(client.get_solvers())

[StructuredSolver(id='DW_2000Q_6'), BQMSolver(id='hybrid_binary_quadratic_model_version2'), DQMSolver(id='hybrid_discrete_quadratic_model_version1'), StructuredSolver(id='Advantage_system1.1')]


In [12]:
if solve_on_remote_dwave_machine:
    #from dwave.cloud.solver import BQMSolver
    #sampler = BQMSolver(client=client, data={id:'hybrid_binary_quadratic_model_version2'})  # id='hybrid_binary_quadratic_model_version2'
    
    from dwave.system import DWaveCliqueSampler
    sampler = DWaveCliqueSampler()    
    sampler.parameters
    print(sampler.parameters)

{'anneal_offsets': ['parameters'], 'anneal_schedule': ['parameters'], 'annealing_time': ['parameters'], 'answer_mode': ['parameters'], 'flux_biases': ['parameters'], 'flux_drift_compensation': ['parameters'], 'h_gain_schedule': ['parameters'], 'initial_state': ['parameters'], 'max_answers': ['parameters'], 'num_reads': ['parameters'], 'num_spin_reversal_transforms': ['parameters'], 'programming_thermalization': ['parameters'], 'readout_thermalization': ['parameters'], 'reduce_intersample_correlation': ['parameters'], 'reinitialize_state': ['parameters'], 'warnings': []}


In [13]:
bqm = model.to_bqm()
sampleset = sampler.sample(bqm, num_reads=100)
decoded_samples = model.decode_sampleset(sampleset)
best_sample = min(decoded_samples, key=lambda f: f.energy)
#best_sample.sample # doctest: +SKIP
#print(best_sample.sample)

In [14]:
vars = list(zip(['b'], [b]))
for (var_name, var) in vars:
    print(var_name + ':')

    line = 'i\\t'
    for (t, (_, _)) in enumerate(cfg['rectangle_types'].items()):
        line += ' {:d}'.format(t)
    print(line)
    
    for i in range(N):
        line = '{:d}  '.format(i)
        for (t, (_, _)) in enumerate(cfg['rectangle_types'].items()):
            s = var_name + '_{:d}_{:d}'.format(i, t)
            val = best_sample.sample[s]
            line += ' {:d}'.format(val)  # is always 0 or 1
        print(line)

vars = list(zip(['x', 'y', 'w', 'h', 's10xL', 's10xR', 's11yL', 's11yR', 's12'],\
                [x, y, w, h, s10xL, s10xR, s11yL, s11yR, s12]))
for (var_name, var) in vars:
    print(var_name + ':')
    
    line = ' \\i '
    for i in range(N):
        line += ' {:3d}'.format(i)
    print(line)

    line = ' ' * (3+1)
    for i, _ in var.items():
        assert isinstance(i, int)
        s = var_name + '_{:d}'.format(i)        
        if var_name in ['s12']:
            val = best_sample.sample[s]
            assert val in [0, 1]
        else:
            val = best_sample.subh[s]
            assert int(val) == val
            assert val >= 0
        line += ' {:3d}'.format(int(val))
    print(line)
            
vars = list(zip(['p', 'q', 's2x', 's3x', 's4y', 's5y'], [p, q, s2x, s3x, s4y, s5y]))
for (var_name, var) in vars:
    print(var_name + ':')

    line = 'i\\j '
    for j in range(N):
        line += ' {:3d}'.format(j)
    print(line)
    
    for i in range(N):
        line = ' {:3d}'.format(i)
        for j in range(N):
            if j<=i:
                line += ' ' * (3+1)
            else:
                s = var_name + '_{:d}_{:d}'.format(i, j)
                if var_name in ['p', 'q']:
                    val = best_sample.sample[s]
                    assert val in [0, 1]
                    line += ' {:3d}'.format(val)  # is always 0 or 1
                else:
                    val = best_sample.subh[s]
                    assert val == int(val)
                    assert val >= 0
                    line += ' {:3d}'.format(int(val))  # is always a positive integer
        print(line)
                

#s12 = {}  # (i) bin

b:
i\t 0
0   1
1   1
x:
 \i    0   1
       3   8
y:
 \i    0   1
       4   4
w:
 \i    0   1
       0   3
h:
 \i    0   1
       0   1
s10xL:
 \i    0   1
       5   9
s10xR:
 \i    0   1
       5  15
s11yL:
 \i    0   1
      26  17
s11yR:
 \i    0   1
       6   2
s12:
 \i    0   1
       0   0
p:
i\j    0   1
   0       1
   1        
q:
i\j    0   1
   0       0
   1        
s2x:
i\j    0   1
   0       2
   1        
s3x:
i\j    0   1
   0       8
   1        
s4y:
i\j    0   1
   0      20
   1        
s5y:
i\j    0   1
   0      25
   1        


In [15]:
best_sample
#decoded_sample = model.decode_sample(best_sample.sample) # vartype='BINARY')

DecodedSample(decoded_subhs=[Constraint(s15_0,energy=1.000000),Constraint(eq15_0,energy=1.000000),Constraint(h_1,energy=1.000000),Constraint(eq14_1,energy=0.040000),Constraint(w_1,energy=3.000000),Constraint(eq13_1,energy=0.640000),Constraint(eq12_1,energy=0.000000),Constraint(y_1,energy=4.000000),Constraint(s11yR_1,energy=2.000000),Constraint(eq11R_1,energy=2.250000),Constraint(s11yL_1,energy=17.000000),Constraint(x_1,energy=8.000000),Constraint(s10xR_1,energy=15.000000),Constraint(eq10R_1,energy=506.250000),Constraint(s10xL_1,energy=9.000000),Constraint(eq10L_1,energy=6.250000),Constraint(h_0,energy=0.000000),Constraint(eq14_0,energy=1.440000),Constraint(w_0,energy=0.000000),Constraint(eq13_0,energy=4.840000),Constraint(eq12_0,energy=0.000000),Constraint(y_0,energy=4.000000),Constraint(s11yR_0,energy=6.000000),Constraint(eq11R_0,energy=25.000000),Constraint(s11yL_0,energy=26.000000),Constraint(x_0,energy=3.000000),Constraint(s10xR_0,energy=5.000000),Constraint(eq10R_0,energy=36.00000

In [16]:
broken_constr_dict = best_sample.constraints(only_broken=True)
n_violated_constraints = len(broken_constr_dict.keys())
print('{:d} constraints are violated.'.format(n_violated_constraints))
broken_constr_dict

15 constraints are violated.


{'eq10L_0': (False, 4.0),
 'eq10L_1': (False, 6.25),
 'eq10R_0': (False, 36.0),
 'eq10R_1': (False, 506.25),
 'eq11R_0': (False, 25.0),
 'eq11R_1': (False, 2.25),
 'eq13_0': (False, 4.840000000000001),
 'eq13_1': (False, 0.6399999999999988),
 'eq14_0': (False, 1.44),
 'eq14_1': (False, 0.040000000000000036),
 'eq15_0': (False, 1.0),
 'eq2_0_1': (False, 42.25),
 'eq3_0_1': (False, 20.25),
 'eq4_0_1': (False, 110.25),
 'eq5_0_1': (False, 420.25)}